In [1]:
import cudf
import dask_cudf

In [2]:
import dask_cudf

In [3]:
from dask_cuml.linear_regression import LinearRegression

In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [4]:
# from dask.distributed import Client
# from dask_cuda import LocalCUDACluster
# cluster = LocalCUDACluster(threads_per_worker = 10)

In [5]:
from dask.distributed import Client
client = Client(scheduler_file = "../cluster.json")
client

Client Scheduler: tcp://10.2.169.115:8786 Dashboard: http://10.2.169.115:8787/status,Cluster Workers: 2 Cores: 2 Memory: 3.15 GB


In [6]:
import numba.cuda

devs = [i.id for i in numba.cuda.cudadrv.devices.gpus]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [7]:
def set_visible(i, n):
    import os, numba.cuda
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
#     numba.cuda.close()
#     numba.cuda.select_device(i)
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd

dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [8]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])


In [9]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

In [10]:
import numba.cuda
import cuml
def print_device(arr):
    print(str(numba.cuda.get_current_device()))
    print(str(cuml.device_of_ptr(arr.as_gpu_matrix(order="F"))))
    
[client.submit(print_device, part) for part in X_df.to_delayed()]

[<Future: status: pending, key: print_device-af84b63dafa485229733d1aa7ab786dc>,
 <Future: status: pending, key: print_device-5bbc7c68b3667dcce87c0821ea07dca1>,
 <Future: status: pending, key: print_device-46a57be1a2c4d11f701df66e0a14b324>,
 <Future: status: pending, key: print_device-6b58c60f59947b526028f2266c5c3fac>]

In [11]:
client.who_has()

{"('from_cudf-5f454479d41448bf9d186c66edd28531', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 3)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 3)": ('tcp://10.2.169.115:35905',),
 'print_device-46a57be1a2c4d11f701df66e0a14b324': ('tcp://10.2.169.115:35895',),
 'print_device-5bbc7c68b3667dcce87c0821ea07dca1': ('tcp://10.2.169.115:35905',),
 'print_device-6b58c60f59947b526028f2266c5c3fac': ('tcp://10.2.169.115:35905',),
 'print_device-af84b63dafa485229733d1aa7ab786dc': ('tcp://10.2.169.115:35895'

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [12]:
client.who_has()

{"('from_cudf-5f454479d41448bf9d186c66edd28531', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-5f454479d41448bf9d186c66edd28531', 3)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 3)": ('tcp://10.2.169.115:35905',),
 'print_device-46a57be1a2c4d11f701df66e0a14b324': ('tcp://10.2.169.115:35895',),
 'print_device-5bbc7c68b3667dcce87c0821ea07dca1': ('tcp://10.2.169.115:35905',),
 'print_device-6b58c60f59947b526028f2266c5c3fac': ('tcp://10.2.169.115:35905',),
 'print_device-af84b63dafa485229733d1aa7ab786dc': ('tcp://10.2.169.115:35895'

In [13]:
lr = LinearRegression()

In [14]:
res = lr.fit(X_df, y_df)

input_devarrays: [(('10.2.169.115', 35905), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-c57804c210b323792f9e3a0e01d80f92>), (('10.2.169.115', 35895), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-8b26cfe89bcc8411e145ecbd28532296>)]
exec_node: ('10.2.169.115', 35905)
ipc_handles: [<Future: status: pending, key: get_input_ipc_handles-141f095edef6e9a07b2f9aaf58e92d2f>]
raw_arrays: [<Future: status: finished, type: tuple, key: inputs_to_device_arrays-c57804c210b323792f9e3a0e01d80f92>]


res

In [15]:
lr.coeffs

(('10.2.169.115', 35905),
 <Future: status: finished, type: DataFrame, key: get_result-3f40cd33d545d433cb9a950f571287a7>)

In [16]:
g = lr.predict(X_df)

WORKER PARTS: [(('10.2.169.115', 35895), <Future: status: finished, type: DataFrame, key: ('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 2)>), (('10.2.169.115', 35905), <Future: status: finished, type: DataFrame, key: ('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 1)>), (('10.2.169.115', 35895), <Future: status: finished, type: DataFrame, key: ('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 0)>), (('10.2.169.115', 35905), <Future: status: finished, type: DataFrame, key: ('from_cudf-d700984431fa4e05b60a88f2f0afe68b', 3)>)]
ON WORKER: 2
NOT ON WORKER: 2
IPCHANDLES = [<Future: status: pending, key: get_ipc_handles-66845a393afb355ca4845c5513775262>, <Future: status: pending, key: get_ipc_handles-0206d11f2d2b9711861ad01e9010e72c>]
RAW_ARRAYS=[<Future: status: pending, key: as_gpu_matrix-ec4d8871c36b13fb9affa8d5474e2b50>, <Future: status: pending, key: as_gpu_matrix-6ae517e1773eafd3c0baad05bf8bcd8b>]
f=<Future: status: finished, type: tuple, key: _predict_on_worker-ac91a12a95b323e1acb5754963e

In [17]:
g

tornado.gen.Return(<Future: status: finished, type: DataFrame, key: get_result-c8643821ba8456feeb528027ec070822>)